In [1]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp

#Custome graph format style sheet
plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = False
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.eps'
if not 'location' in locals():
    #save location.  First one is for running on home PC, second for running on the work laptop.  May need to make a global change
    location = 'E:\\Documents\\Dan\\Code\\FigsAndPlots\\FigsAndPlotsDocument\\Figures\\'
    #location = 'C:\\Users\\dhendrickson\\Documents\\github\\FigsAndPlots\\FigsAndPlotsDocument\\Figures\\'

my_cmap = plt.get_cmap('gray')
#Standard cycle for collors and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '--', ':', '-.']))
plt.rc('axes', prop_cycle=default_cycler)

In [2]:
#special Header for package imports:
import os
from random import random
import datetime

import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
 
import cv2
from sklearn.model_selection import train_test_split
import keras_metrics as km
 
 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix

import tensorflow as tf

## Logic for this program:
1. Choose Random set from Repository
1. Parse Name to get accepted output vector
1. Train Models on data
    * Data Prep:
      * Try un smoothed
      * Try smoothed with rolling average
      * try smooth with kalman
      * keep data trunkated with smoothing
    * Model TypesVideo and Image
      * Video:
        * Rolling, 3x100, 3x changing frame sizes
        * overlapping segments, 3x?? hoping down line
        * rolling fingerprints
          * Try many fingerprints
          * look at Beta, and classification as it is going
      * Image:
        * Fingerprint
        * 3 x 60k image
1. Examine differences of results

In [3]:
SensorPositonFile = 'C:\\Users\\dhendrickson\\Documents\\Code\\RMG_NDE_Disssertation\\SensorLocations.csv'
SensorPositonFile = 'C:\\Users\\danhe\\Code\\RMG_NDE_Disssertation\\SensorLocations.csv'
OutputVectors = np.genfromtxt(open(SensorPositonFile,'r'), delimiter=',',skip_header=1,missing_values=0)

def truthVector(Filename):
    # Parses the filename, and compares it against the record of sensor position on cranes
    # inputs: filename
    # outputs: truth vector


    #Parsing the file name.  Assuming it is in the standard format
    sYear = '20' + Filename[10:12]
    sMonth = Filename[12:14]
    sDay = Filename[14:16]
    sHour = Filename[17:19]
    sMin = Filename[19:21]
    sSensor = Filename[23]

    mask = []

    i=0
    #loops through the known sensor movements, and creates a filter mask
    for spf in OutputVectors:
        
        startDate = datetime.date(int(spf[0]), int(spf[1]), int(spf[2])) + datetime.timedelta(hours=spf[3]) + datetime.timedelta(minutes=spf[4])
        endDate = datetime.date(int(spf[5]), int(spf[6]), int(spf[7])) + datetime.timedelta(hours=spf[8]) + datetime.timedelta(minutes=spf[9])
        sDate = datetime.date(int(sYear), int(sMonth), int(sDay)) + datetime.timedelta(hours=int(sHour)) + datetime.timedelta(minutes=int(sMin))

        if sDate >= startDate and sDate <= endDate and int(spf[10]) == int(sSensor):
            mask.append(True)
            i+=1
        else:
            mask.append(False)
        
    if i ==0: print('error ', Filename)

    results = OutputVectors[mask,11:]

    #np.array(results)

    return results

def makeFrames(input,sequ,frameLength):
    frames=[] #np.array([],dtype=object,)
    segmentGap = int((np.shape(input)[0]-frameLength)/sequ)
    #print(segmentGap,sequ, frameLength)
    for i in range(segmentGap):
        start = i * segmentGap
        imageMatrix = input[start:start+frameLength,:]
        np.matrix(imageMatrix)
        frames.append(imageMatrix)
    
    return frames

In [4]:
# Make sample data set with labels

#Number of files to get for test
n = 10
img_height , img_width = 3, 100
seq_len = 600

#folder = 'E:\\Documents\\Dan\\PhD\\Data Backup\\ASC Accel Pi\\Excel Versions\\'
folder = 'C:\\Users\\danhe\\Code\\TestData\\'
files = os.listdir(folder)
#files = random.sample(files,n)

DataSet = [] #np.array([],dtype=object,)

ResultsSet = np.zeros((len(files),np.shape(OutputVectors[:,11:])[1])) #np.array([],dtype=object,)

    

In [5]:
np.shape(ResultsSet)

(91, 44)

In [6]:
i=0
for filename in files:
    ResultsSet[i,:] = truthVector(filename)
    fileData = np.genfromtxt(open(folder+filename,'r'), delimiter=',',skip_header=0,missing_values=0).T[2:5,:]
    frames = makeFrames(fileData.T,seq_len,img_width)
    #print(np.shape(DataSet),np.shape(frames),1)
    #frames = np.asarray(frames)
    DataSet.append(frames)
    i+=1

#ResultsSet = np.asarray(ResultsSet)
#DataSet = np.asarray(DataSet)



In [7]:
np.shape(DataSet)


(91, 99, 100, 3)

In [8]:
np.shape(ResultsSet)


(91, 44)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(DataSet, ResultsSet, test_size=0.20, shuffle=True, random_state=0)


In [10]:
np.shape(X_train)

(72, 99, 100, 3)

In [14]:
np.shape(y_train)[1]

44

In [12]:
k = np.shape(X_train[0][0][:][:])

for i in range(np.shape(X_train)[0]):
    for j in range(np.shape(X_train)[1]):
        if np.shape(X_train[i][j][:][:]) != k:
            print(i,j,np.shape(X_train[i][j][:][:]))
        k = np.shape(X_train[i][j][:][:])



In [13]:
model = Sequential()
model.add(ConvLSTM2D(filters = 64, 
            kernel_size = (3, 3), 
            return_sequences = False, 
            data_format = "channels_last", 
            input_shape = (seq_len, img_width, img_height,1)
            )
        )
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(np.shape(y_train)[1], activation = "softmax"))
 
model.summary()
 
opt = tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
 
earlystop = EarlyStopping(patience=7)   
callbacks = [earlystop]
 
history = model.fit(x = X_train, y = y_train, epochs=40, batch_size = 8 , shuffle=True, validation_split=0.2, callbacks=callbacks)
 
y_pred = np.argmax(y_pred, axis = 1)
y_test = np.argmax(y_test, axis = 1)
 
print(classification_report(y_test, y_pred))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 98, 1, 64)         150016    
                                                                 
 dropout (Dropout)           (None, 98, 1, 64)         0         
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 256)               1605888   
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 44)                11308     
                                                                 
Total params: 1,767,212
Trainable params: 1,767,212
Non-

ValueError: Data cardinality is ambiguous:
  x sizes: 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80